In [2]:
import requests

In [4]:

AZURE_API_KEY = '5f3dfecbd34d4bed939cd0e4b7abed63'  # Replace with your actual API key
AZURE_ENDPOINT = 'https://prof-insight-ai-service.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-03-15-preview'

def get_azure_response(user_input):
    headers = {
        'Content-Type': 'application/json',
        'api-key': AZURE_API_KEY
    }
    
    # Constructing the data payload for chat models
    data = {
        "messages": [
            {"role": "user", "content": user_input}
        ],
        "max_tokens": 500  # Control response length
    }
    
    response = requests.post(AZURE_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        response_json = response.json()
        return response_json['choices'][0]['message']['content']  # Get response text
    else:
        return f"Error: {response.status_code}, {response.text}"



I'm just a computer program, so I don't have feelings, but thanks for asking! How can I assist you today?


In [5]:
user_input = 'What does Axel do generally'
response = get_azure_response(user_input)
print(response)

Axel is a name that can refer to different individuals in various contexts, each possibly having distinct roles or professions. Without additional information, it's challenging to pinpoint what a specific "Axel" does. However, I can provide some general examples:

1. **Axel in Literature or Media**: There could be characters named Axel in books, movies, or TV shows who have specific roles or story arcs. For instance, Axel Foley is a famous character played by Eddie Murphy in the "Beverly Hills Cop" movie series, where he is a detective.

2. **Axel in Technology or Business**: There might be individuals named Axel in the tech or business world who could be CEOs, developers, entrepreneurs, or any other professional role.

3. **Axel in Sports**: This could be in reference to athletes, such as professional skaters or soccer players.

4. **Axel in Music**: Axel could be the name of musicians, composers, or singers.

5. **Axel in Everyday Life**: Axel might be an ordinary person's first name

In [10]:
import requests
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Azure Search configuration
search_service_endpoint = "https://prof-insight-search.search.windows.net"
index_name = "prof-insight-index"
api_key = "9K3QkXHzTrk6kPP78fjug3NSG3U93HQMK28pEXF5oLAzSeDWVa3i"

# Create Azure Search client
search_client = SearchClient(endpoint=search_service_endpoint,
                             index_name=index_name,
                             credential=AzureKeyCredential(api_key))

# Azure OpenAI configuration
AZURE_API_KEY = '5f3dfecbd34d4bed939cd0e4b7abed63'
AZURE_ENDPOINT = 'https://prof-insight-ai-service.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-03-15-preview'

def retrieve_context(user_input):
    results = search_client.search(search_text=user_input)
    relevant_contexts = [result['text'] for result in results]  # Adjust 'content' to your actual field
    return "\n".join(relevant_contexts) if relevant_contexts else "No relevant context found."

def get_azure_response(user_input, context):
    headers = {
        'Content-Type': 'application/json',
        'api-key': AZURE_API_KEY
    }
    
    # Combine context with user input
    combined_input = f"Context:\n{context}\n\nUser: {user_input}"
    
    data = {
        "messages": [
            {"role": "user", "content": combined_input}
        ],
        "max_tokens": 500
    }
    
    response = requests.post(AZURE_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        response_json = response.json()
        return response_json['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code}, {response.text}"



In [13]:
user_input = "How are you?" 
relevant_context = search_client.search(search_text=user_input)

In [14]:
type(relevant_context)

azure.search.documents._paging.SearchItemPaged

In [11]:
# Main logic

 # Example user input
context = retrieve_context(user_input)
response = get_azure_response(user_input, context)
print(f"Prof-insight:\n{response}")


KeyError: 'text'

In [16]:
from azure.search.documents import SearchClient, SearchIndexClient
from azure.core.credentials import AzureKeyCredential

# Azure Search configuration
search_service_endpoint = "https://prof-insight-search.search.windows.net"
api_key = "9K3QkXHzTrk6kPP78fjug3NSG3U93HQMK28pEXF5oLAzSeDWVa3i"
index_name = "prof-insight-index"

# Create an index client
index_client = SearchIndexClient(endpoint=search_service_endpoint, credential=AzureKeyCredential(api_key))

def list_index_fields(index_name):
    index = index_client.get_index(index_name)
    print("Fields in index:")
    for field in index.fields:
        print(f"- {field.name} (type: {field.type})")

# Call the function to list fields
list_index_fields(index_name)


ImportError: cannot import name 'SearchIndexClient' from 'azure.search.documents' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/azure/search/documents/__init__.py)

In [17]:
import os
import openai
import dotenv

In [26]:
dotenv.load_dotenv()

endpoint = 'https://prof-insight-ai-service.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-03-15-preview'
api_key = '5f3dfecbd34d4bed939cd0e4b7abed63'
deployment = 'gpt-4o'

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

In [27]:
completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "What are my available health plans?",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": 'https://prof-insight-search.search.windows.net',
                    "index_name": 'prof-insight-index',
                    "authentication": {
                        "type": "api_key",
                        "key": 'os.environ["AZURE_AI_SEARCH_API_KEY"]',
                    }
                }
            }
        ],
    }
)


In [28]:
print(completion.model_dump_json(indent=2))

{
  "id": "53895d84-9e08-47e1-be7c-62b81c6def03",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The requested information is not available in the retrieved data. Please try another query or topic.",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null,
        "end_turn": true,
        "context": {
          "citations": [
            {
              "content": "In the original version of this Article, the affiliation details for Jadranka Loncarek and Vito Mennella were incorrectly given as 'Cell Biology Program, The Hospital for Sick Children, Department of Biochemistry, University of Toronto, 555 University Avenue, Toronto, ON, M5G 1X8, Canada' and 'Laboratory of Protein Dynamics and Signaling, Center for Cancer Research, National Cancer Institute, 1050 Boyles Street, Frederick, MD, 21702, USA', respectively. This has now been corrected in 